<a href="https://colab.research.google.com/github/danielsoy/ADer/blob/main/colab_retrainable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-


In [ ]:
!git clone https://github.com/OpenAOI/anodet.git

In [ ]:
cd /content/anodet

In [12]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 779.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [13]:
import os
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from torchvision import transforms
from tqdm import tqdm
#import anodet
from torch.optim.lr_scheduler import MultiStepLR

In [17]:
cd /content/anodet/anodet

/content/anodet/anodet


In [15]:
import anodet

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
local=False # train model with local dataset if true. train from colab mydrive if not.

In [20]:
if local:
    # Setup local paths
    DATASET_PATH = r"C:\anodet\notebooks\video_pro\working folder\indexed4augmet\datasets\DATASET A"
    MODEL_SAVE_PATH = "./trained_backbone.pth"
else:
    # Setup mydrive colab paths
    DATASET_PATH    = os.path.realpath("/content/drive/MyDrive/DATASET A")
    MODEL_SAVE_PATH = os.path.realpath("/content/drive/MyDrive/DATASET A MODEL")

In [21]:
IMAGE_SIZE = 224  # Original ResNet training size
BATCH_SIZE = 16  # Fixed batch size


Setup device
<br>
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')<br>
print(f"Using device: {device}")<br>
# /content/anodet/anodet/datasets/dataset.py<br>
ustom dataset with resizing


In [22]:
cd /content/anodet

/content/anodet


rom anodet import anodet

rom anodet.anodet import datasets

rom anodet.anodet.datasets import dataset

In [23]:
from anodet.anodet.datasets.dataset import AnodetDataset
#    folder floder floder   .py script     clase.

In [24]:
class ResNetDataset(AnodetDataset):
    def __init__(self, root_dir):
        super().__init__(root_dir)
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),  # Standard ResNet size
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],  # ImageNet normalization
                std=[0.229, 0.224, 0.225]
            )
        ])
    def __getitem__(self, index):
        img, label, mask = super().__getitem__(index)
        if isinstance(img, np.ndarray):
            img = self.transform(img)
        return img, label, mask

In [25]:
def calculate_feature_loss(features):
    loss = 0
    weights = [0.1, 0.2, 0.3, 0.4]
    for feat, weight in zip(features, weights):
        b, c, h, w = feat.shape
        feat_reshaped = feat.view(b, c, -1)
        feat_norm = feat_reshaped / (torch.norm(feat_reshaped, dim=2, keepdim=True) + 1e-10)
        similarity = torch.bmm(feat_norm, feat_norm.transpose(1, 2))
        target = torch.eye(c, device=device).unsqueeze(0).expand(b, -1, -1)
        loss += weight * torch.mean((similarity - target) ** 2)
    return loss

In [26]:
def train_resnet_backbone(
    dataloader: DataLoader,
    num_epochs: int = 200,  # Original ResNet epochs
    initial_lr: float = 0.075,  # Original ResNet learning rate
    continue_training: bool = False  # New parameter
):
    model = resnet18(weights=None)

    # Load previous model if continuing training
    if continue_training:
        try:
            # Try to load the best model first
            model.load_state_dict(torch.load(MODEL_SAVE_PATH + '.best'))
            print(f"Loaded best model from {MODEL_SAVE_PATH}.best")
        except:
            # Fall back to the last saved model
            model.load_state_dict(torch.load(MODEL_SAVE_PATH))
            print(f"Loaded last saved model from {MODEL_SAVE_PATH}")

    model = model.to(device)

    # Original ResNet optimizer settings
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=initial_lr,
        momentum=0.9,
        weight_decay=0.0001
    )

    # Original ResNet learning rate schedule
    milestones = [30, 60]  # Original ResNet milestones
    scheduler = MultiStepLR(  # Fixed MultiStepLR usage
        optimizer,
        milestones=milestones,
        gamma=0.1
    )
    best_loss = float('inf')
    patience_counter = 0
    print("Training settings:")
    print(f"Image size: {IMAGE_SIZE}x{IMAGE_SIZE}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Initial LR: {initial_lr}")
    print(f"LR milestones: {milestones}")
    print(f"Epochs: {num_epochs}")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, _, _ in tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            images = images.to(device)

            # Extract features
            features = []
            x = model.conv1(images)
            x = model.bn1(x)
            x = model.relu(x)
            x = model.maxpool(x)
            x = model.layer1(x)
            features.append(x)
            x = model.layer2(x)
            features.append(x)
            x = model.layer3(x)
            features.append(x)
            x = model.layer4(x)
            features.append(x)
            loss = calculate_feature_loss(features)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        epoch_loss = running_loss / len(dataloader)
        current_lr = optimizer.param_groups[0]['lr']
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.6f}, LR: {current_lr:.6f}')
        scheduler.step()
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            patience_counter = 0
            print(f"New best loss: {best_loss:.6f}")
            torch.save(model.state_dict(), MODEL_SAVE_PATH + '.best')
        else:
            patience_counter += 1
            if patience_counter >= 20:  # Increased patience
                print("Early stopping triggered")
                break
    return model

In [ ]:
# set variable:continue_training=True if .best model already exist in mydrive.
# y set device to T4 gpu, not CPU.

In [27]:
if __name__ == "__main__":
    # Create dataset with fixed batch size
    dataset = ResNetDataset(DATASET_PATH)
    dataloader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,  # Using fixed batch size
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    print(f"Number of training images: {len(dataset)}")

    # To continue training from saved model:
    trained_model = train_resnet_backbone(
        dataloader,
        continue_training=True  # Set to True to load previous model
    )

    # Save the final model
    print(f"Saving model to {MODEL_SAVE_PATH}")
    torch.save(trained_model.state_dict(), MODEL_SAVE_PATH)
    print("Training completed!")

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Number of training images: 5472


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.


<br>
###############################<br>
trained_model = train_resnet_backbone(dataloader, continue_training=False)<br>
trained_model = train_resnet_backbone(dataloader,  continue_training=True)<br>
###############################<br>
